# WS24 - Intelligente Informationssysteme

## Block 3: Retrieval Augmented Generation

Build your first simple RAG with LangChain. We follow the LangChain Tutorial "Build a Retrieval Augmented Generation (RAG) App" found at <https://python.langchain.com/docs/tutorials/rag/>.

**Part 2: Retrieve Knowledge from Vectorstore and Generate Answers**



In [1]:
# Langchain
# Vector stores are commonly used for retrieval, but there are other ways to do retrieval, too.
# Retriever: An object that returns Documents given a text query
# Open a vectore store as retriever
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

from langchain_chroma import Chroma
vectorstore = Chroma(persist_directory="vector_store", collection_name="lils_blogs", embedding_function=embeddings)

vectorstore

In [2]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

question = "What are the approaches to task decomposition?"

retrieved_docs = retriever.invoke(question)

len(retrieved_docs)

6

In [5]:
print(retrieved_docs[0].page_content)

Fig. 1. Overview of a LLM-powered autonomous agent system.
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.


In [6]:
## Chat Models: Modern LLMs are typically accessed through a chat model interface 
# that takes a list of messages as input and returns a message as output.
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:latest")

In [7]:
# LangChain Prompt
from langchain_core.prompts import PromptTemplate

prompt = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

prompt_template = PromptTemplate.from_template(prompt)

In [8]:
messages = prompt_template.invoke(
    {"context": retrieved_docs[0].page_content, "question": question}
).to_messages()
messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: What are the approaches to task decomposition? \nContext: Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process. \nAnswer:", additional_kwargs={}, response_metadata={})]

In [9]:
message = llm.invoke(messages)

In [10]:
print(type(message))
print(message.content)
print("---------------")
print("id               :", message.id)
print("additional_kwargs:", message.additional_kwargs)
print("response_metadata:", message.response_metadata)
print("usage_metadata   :", message.usage_metadata)


<class 'langchain_core.messages.ai.AIMessage'>
Task decomposition approaches include Chain of Thought (CoT), which involves instructing a model to break down complex tasks into smaller, simpler steps by "thinking step by step". Another approach is not explicitly mentioned in the provided context. CoT has become a standard prompting technique for enhancing model performance on complex tasks.
---------------
id               : run-d4a4c5d9-5629-4ac6-b004-f28e4f1b3f9f-0
additional_kwargs: {}
response_metadata: {'model': 'llama3.2:latest', 'created_at': '2024-11-30T10:53:46.403054Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4912976792, 'load_duration': 822084875, 'prompt_eval_count': 214, 'prompt_eval_duration': 3249000000, 'eval_count': 62, 'eval_duration': 840000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}
usage_metadata   : {'input_tokens': 214, 'output_tokens': 62, 'total_tokens': 276}
